In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
tond = pd.read_csv("type_before_pre_ton.csv")

In [ ]:
iot20 = pd.read_csv("iot2020.csv")

In [ ]:
#5:5 비율을 맞추기 위해 ton-iot의 개수만큼 랜덤 추출
iot20 = iot20.sample(n=461043, replace=False, axis=0)

In [ ]:
del tond['Unnamed: 0'] 

In [ ]:
tond['type'].value_counts()

In [ ]:
#feature명 통일
tond.rename(columns={'type':'Cat'})

In [ ]:
#공격명 통일
iot20 = iot20.replace('Normal', 'normal' )
iot20 = iot20.replace('DoS', 'dos' )
iot20 = iot20.replace('MITM ARP Spoofing', 'mitm' )
iot20 = iot20.replace('Scan', 'scanning' )

In [ ]:
iot20 = pd.get_dummies(iotd, columns=['conn_state'])  

In [ ]:
iot20 = iot20.drop(columns=[ 'Label','Sub_Cat','Flow_ID', 'Src_IP', 'Dst_IP',  'Timestamp', 'Src_IP', 'Src_Port', 'Dst_IP', 'Dst_Port'])

In [ ]:
tond = tond.drop(columns=[ 'ts', 'src_port', 'dst_port'])

In [ ]:
iot20.info()

In [ ]:
#ton에 없는 feature명 추가
tond = tond.reindex(columns = tond.columns.tolist() +['Flow_IAT_Max', 'Flow_IAT_Min', 'SYN_Flag_Cnt', 'Bwd_IAT_Mean', 'Init_Fwd_Win_Byts', 'Fwd_IAT_Std', 'Pkt_Size_Avg', 'Active_Min', 'Fwd_Act_Data_Pkts', 'Active_Mean', 'Bwd_IAT_Max', 'Bwd_Pkt_Len_Min', 'PSH_Flag_Cnt', 'TotLen_Fwd_Pkts', 'Subflow_Fwd_Byts', 'FIN_Flag_Cnt', 'Active_Max', 'Fwd_Pkt_Len_Min', 'URG_Flag_Cnt', 'Subflow_Bwd_Byts', 'Bwd_Pkt_Len_Max', 'Subflow_Fwd_Pkts', 'Bwd_IAT_Std', 'Bwd_Pkt_Len_Std', 'Flow_IAT_Mean', 'Subflow_Bwd_Pkts', 'Bwd_Byts/b_Avg', 'Pkt_Len_Min', 'RST_Flag_Cnt', 'Idle_Std', 'Fwd_Seg_Size_Min', 'Fwd_URG_Flags', 'Flow_Pkts/s', 'ACK_Flag_Cnt', 'Fwd_Pkts/b_Avg', 'Bwd_PSH_Flags', 'Bwd_Seg_Size_Avg', 'Bwd_Header_Len', 'Pkt_Len_Mean', 'Init_Bwd_Win_Byts', 'Pkt_Len_Std', 'Fwd_Pkt_Len_Max', 'Bwd_IAT_Min', 'Bwd_IAT_Tot', 'Pkt_Len_Var', 'Fwd_Pkt_Len_Std', 'TotLen_Bwd_Pkts', 'Pkt_Len_Max', 'Flow_Byts/s', 'Bwd_URG_Flags', 'ECE_Flag_Cnt', 'Fwd_IAT_Min', 'Fwd_Blk_Rate_Avg', 'Fwd_PSH_Flags', 'Down/Up_Ratio', 'Bwd_Pkts/b_Avg', 'CWE_Flag_Count', 'Idle_Mean', 'Fwd_IAT_Tot', 'Fwd_Seg_Size_Avg', 'Flow_IAT_Std', 'Fwd_Header_Len', 'Idle_Max', 'Fwd_IAT_Max', 'Idle_Min', 'Fwd_IAT_Mean', 'Bwd_Blk_Rate_Avg', 'Active_Std'])

In [ ]:
#iot20에 없는 feature명 추가
iot20 = iot20.reindex(columns = iot20.columns.tolist() + ['dns_AA', 'http_resp_mime_types', 'service', 'ssl_cipher', 'http_orig_mime_types', 'weird_notice', 'http_status_code', 'dns_qtype', 'ssl_subject', 'ssl_resumed', 'http_response_body_len', 'dns_RA', 'http_request_body_len', 'ssl_issuer', 'ssl_established', 'dns_rcode', 'weird_name', 'dns_RD', 'weird_addl', 'dns_rejected', 'ssl_version', 'dns_qclass', 'http_version', 'http_trans_depth', 'conn_state', 'missed_bytes', 'http_method'])

In [ ]:
#feature명 맞추기
tond= tond.rename(columns={'duration':'Flow_Duration','src_pkts':'Tot_Fwd_Pkts', 'dst_pkts':'Tot_Bwd_Pkts', 'proto':'Protocol', 'src_bytes':'Fwd_Pkt_Len_Mean' , 'dst_bytes':'Bwd_Pkt_Len_Mean', 'src_ip_bytes':'Fwd_Pkts/s', 'dst_ip_bytes':'Bwd_Pkts/s', 'type':'Cat', 'ts':'Timestamp', 'src_port':'Src_Port', 'dst_port':'Dst_Port', 'label':'Label'})

In [ ]:
#필요없는 feature 드랍
tond = tond.drop(columns=[ 'Label', 'Sub_Cat','Flow_ID', 'Src_IP', 'Dst_IP',  'Timestamp'])

In [ ]:
iot20['Cat'].value_counts()

In [ ]:
tond['Cat'].value_counts()

In [ ]:
print(df_merge.columns.tolist())

In [ ]:
df_merge = df_merge.drop(columns=['Label'])

In [ ]:
#데이터 합치기
frames = [iot20, tond]
df_merge = pd.concat(frames)

In [ ]:
#결측값 존재 여부 확인
df_merge.isna().sum()

In [ ]:
#결측값 채우기
tond.fillna(-1,inplace=True)
iot20.fillna(-1,inplace=True)
df_merge.fillna(-1,inplace=True)

In [ ]:
X = df_merge[['dns_AA', 'http_resp_mime_types', 'service', 'ssl_cipher', 'http_orig_mime_types', 'weird_notice', 'http_status_code', 'dns_qtype', 'ssl_subject', 'ssl_resumed', 'http_response_body_len', 'dns_RA', 'http_request_body_len', 'ssl_issuer', 'ssl_established', 'dns_rcode', 'weird_name', 'dns_RD', 'weird_addl', 'dns_rejected', 'ssl_version', 'dns_qclass', 'http_version', 'http_trans_depth', 'conn_state', 'missed_bytes', 'http_method','Protocol', 'Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Min', 'Fwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Mean', 'Bwd_Pkt_Len_Std', 'Flow_Byts/s', 'Flow_Pkts/s', 'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Tot', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min', 'Bwd_IAT_Tot', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Len', 'Bwd_Header_Len', 'Fwd_Pkts/s', 'Bwd_Pkts/s', 'Pkt_Len_Min', 'Pkt_Len_Max', 'Pkt_Len_Mean', 'Pkt_Len_Std', 'Pkt_Len_Var', 'FIN_Flag_Cnt', 'SYN_Flag_Cnt', 'RST_Flag_Cnt', 'PSH_Flag_Cnt', 'ACK_Flag_Cnt', 'URG_Flag_Cnt', 'CWE_Flag_Count', 'ECE_Flag_Cnt', 'Down/Up_Ratio', 'Pkt_Size_Avg', 'Fwd_Seg_Size_Avg', 'Bwd_Seg_Size_Avg', 'Fwd_Pkts/b_Avg', 'Fwd_Blk_Rate_Avg', 'Bwd_Byts/b_Avg', 'Bwd_Pkts/b_Avg', 'Bwd_Blk_Rate_Avg', 'Subflow_Fwd_Pkts', 'Subflow_Fwd_Byts', 'Subflow_Bwd_Pkts', 'Subflow_Bwd_Byts', 'Init_Fwd_Win_Byts', 'Init_Bwd_Win_Byts', 'Fwd_Act_Data_Pkts', 'Fwd_Seg_Size_Min', 'Active_Mean', 'Active_Std', 'Active_Max', 'Active_Min', 'Idle_Mean', 'Idle_Std', 'Idle_Max', 'Idle_Min' ]]
y = df_merge['Cat'] 

labels = X.columns

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X, y)

importance = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
indices = np.argsort(importance)[::-1]

print("Feature ranking:")

for f in range(X.shape[1]):
    print("{}. {} ({:.5f})".format(f + 1, X.columns[indices][f], importance[indices[f]]))